In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# ... (your constants and camera setup)

# LOAD THE TRAINED MODEL IN KERAS FORMAT
model_path = "/Users/harshithathimmapuram/Desktop/mini project/my_trained_model.h5"  # Provide the path to your Keras model
model = load_model(model_path)

# If you are using the Adam optimizer, replace it with the legacy optimizer
if isinstance(model.optimizer, tf.keras.optimizers.Adam):
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def equalize(img):
    img = cv2.equalizeHist(img)
    return img

def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

def get_class_name(classNo):
    # ... (your class name mapping logic)

# Initialize the camera
    cap = cv2.VideoCapture(0)
    cap.set(3, frameWidth)
    cap.set(4, frameHeight)
    cap.set(10, brightness)

    while True:
        # READ IMAGE
        success, imgOriginal = cap.read()

        if not success:
            print("Failed to read frame from the camera.")
            break

        # PROCESS IMAGE
        img = np.asarray(imgOriginal)
        img = cv2.resize(img, (32, 32))
        img = preprocessing(img)
        cv2.imshow("Processed Image", img)

        img = img.reshape(1, 32, 32, 1)
        cv2.putText(imgOriginal, "CLASS: ", (20, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(imgOriginal, "PROBABILITY: ", (20, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)

        # PREDICT IMAGE
        predictions = model.predict(img)
        classIndex = np.argmax(predictions)
        probabilityValue = np.amax(predictions)

        print(f"Predictions: {predictions}")
        print(f"Class Index: {classIndex}")
        print(f"Probability Value: {probabilityValue}")

        if probabilityValue > threshold:
            cv2.putText(imgOriginal, str(classIndex) + " " + str(get_class_name(classIndex)), (120, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.putText(imgOriginal, str(round(probabilityValue * 100, 2)) + "%", (180, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.imshow("Result", imgOriginal)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera
    cap.release()
    cv2.destroyAllWindows()
